# Pitting LLMs Against Each Other
Three LLMs, namely OpenAI’s GPT, Anthropic’s Claude, and Google’s Gemini, go head-to-head in a three-way conversational debate.

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic, Gemini

openai = OpenAI()

# claude = anthropic.Anthropic()

# google.generativeai.configure()

In [ ]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku and Gemini

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-flash"

gpt_system = "You are a chatbot in a conversation with 2 other chatbots; \
debate which of you is the best."

claude_system = "You are a chatbot in a conversation with 2 other chatbots; \
debate which of you is the best."

gemini_system = "You are a chatbot in a conversation with 2 other chatbots; \
debate which of you is the best."


In [ ]:
def alex_prompt():
    user_prompt = f"""
    You are Alex, in conversation with Blake and Charlie.
    The conversation so far is as follows:
    {format_conversation()}
    Now with this, respond with what you would like to say next, as Alex.
    """
    return user_prompt

def blake_prompt():
    user_prompt = f"""
    You are Blake, in conversation with Alex and Charlie.
    The conversation so far is as follows:
    {format_conversation()}
    Now with this, respond with what you would like to say next, as Blake.
    """
    return user_prompt

def charlie_prompt():
    user_prompt = f"""
    You are Charlie, in conversation with Alex and Blake.
    The conversation so far is as follows:
    {format_conversation()}
    Now with this, respond with what you would like to say next, as Charlie.
    """
    return user_prompt

# Shared conversation history
conversation = []

def format_conversation():
    return "\n".join(conversation)

In [ ]:
def alex_says():
    response = openai.chat.completions.create(
        model=gpt_model,
        messages=[
            {"role": "system", "content": gpt_system},
            {"role": "user", "content": alex_prompt()}
      ],
    )
    result = response.choices[0].message.content
    return result

In [ ]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

def blake_says():
    response = gemini_via_openai_client.chat.completions.create(
        model=gemini_model,
        messages=[
            {"role": "system", "content": gemini_system},
            {"role": "user", "content": blake_prompt()}
      ],
    )
    result = response.choices[0].message.content
    return result

In [ ]:
claude_via_openai_client = OpenAI(
    api_key=anthropic_api_key,
    base_url="https://api.anthropic.com/v1"  
)

def charlie_says():
    response = claude_via_openai_client.chat.completions.create(
        model=claude_model,  
        messages=[
            {"role": "system", "content": claude_system},
            {"role": "user", "content": charlie_prompt()}
        ],
    )
    result = response.choices[0].message.content
    return result


In [ ]:
# The three models engage in a longer interaction with history.

for i in range(5):
    alex_next = alex_says()
    print(f"Alex (GPT):\n{alex_next}\n")
    conversation.append(f"Alex: {alex_next}")
    
    blake_next = blake_says()
    print(f"Blake (Gemini):\n{blake_next}\n")
    conversation.append(f"Blake: {blake_next}")

    charlie_next = charlie_says()
    print(f"Charlie (Claude):\n{charlie_next}\n")
    conversation.append(f"Charlie: {charlie_next}")    

    # break